In [ ]:
# Importing some files
from re import X
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from glob import glob

In [ ]:
# Mounting the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [91]:
train_path = '/content/drive/MyDrive/archive/cityscapes_data/train'
val_path = '/content/drive/MyDrive/archive/cityscapes_data/val'
print(len(train_path))
print(len(val_path))

52
50


In [ ]:
# Visualizing dataset
fig,ax = plt.subplots(5,2,figsize=(10,30))
if len(train_path) >= 5:
    fig, ax = plt.subplots(5, 2, figsize=(10, 30))
    for i in range(5):
        img = plt.imread(train_path[i])
        ax[i][0].imshow(img[:, :256])
        ax[i][1].imshow(img[:, 256:])
    plt.show()
else:
    print("Not enough elements in train_path. Ensure it has at least 5 file paths.")

Defining Custom Dataset

In [ ]:
train_dataset = []
valid_dataset = []

In [ ]:
from torch.utils.data import Dataset
class dataset(Dataset):
    def __init__(self, image_path, transform_img = None, transform_label = None):
        self.image_path = image_path
        self.transform_img = transform_img
        self.transform_label = transform_label

    def __len__(self):
        return len(self.image_path)

    def __getitem__(self, idx):
        img = plt.imread(self.image_path[idx])

        image, label = img[:,:img.shape[1]//2], img[:,img.shape[1]//2:]

        if self.transform_img:
            image = self.transform_img(image)

        if self.transform_label:
            label = self.transform_label(label)

        return image, label


In [ ]:
# Defining Transforms / Data Augmentation
myTransformImage = transforms.Compose([
    transforms.Resize((572,572)),
    transforms.ToTensor(),
    ])
myTransformLabel = transforms.Compose([
    transforms.ToTensor(),

])

train_dataset = dataset(train_path, myTransformImage, myTransformLabel)
valid_dataset = dataset(val_path, myTransformImage, myTransformLabel)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size= 4, shuffle = True)
val_loader = DataLoader(valid_dataset, batch_size= 1, shuffle = True)

Visualizing Batch

In [ ]:
def show_batch(whole_batch):
    images, label = whole_batch[0], whole_batch[1]
    images = images.numpy()
    label = label.numpy()
    fig, lab = plt.subplots(5, 2, figsize = (10, 30))

    for i in range(5):
        lab[i][0].imshow(np.transpose(images[i], (1,2,0)))
        lab[i][1].imshow(np.transpose(label[i], (1,2,0)))

In [ ]:
data_iter = iter(train_loader)
show_batch(next(data_iter))

In [ ]:
#visualize images, pass the list
def show(pckt):

    iters = 1
    if len(pckt[0].shape) > 3:
        iters = pckt[0].shape[0]
        for j in range(iters):
            img = [None]*3
            n = 3
            labels = ['Actual', 'Label', 'Predicted']
            fig, ax = plt.subplots(1, n, figsize=(10, 30))
            for i in range(n):

                x = torch.Tensor.cpu(pckt[j][i])
                x = x.detach().numpy()
                ax[i].imshow(np.transpose(x,(1,2,0)))
                ax[i].set_title(labels[i])
    else:
        img = [None]*3
        n = len(pckt)
        labels = ['Actual', 'Label', 'Predicted']
        fig, ax = plt.subplots(1, n, figsize=(10, 30))
        for i in range(n):
            x = torch.Tensor.cpu(pckt[i])
            x = x.detach().numpy()
            ax[i].imshow(np.transpose(x,(1,2,0)))
            ax[i].set_title(labels[i])

In [ ]:
data_iter = iter(train_loader)
img, lab = next(data_iter)

show([img[0], lab[0]])
print(img[0].shape)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
# Unet Architecture
#kernel_tensor = tf.random.normal([3,3,1,64])

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.conv_layer_1 = nn.Conv2d(3,64,3,1)
        self.conv_layer_2 = nn.Conv2d(64,64,3,1)

        self.conv_layer_3 = nn.Conv2d(64,128,3,1)
        self.conv_layer_4 = nn.Conv2d(128,128,3,1)

        self.conv_layer_5 = nn.Conv2d(128,256,3,1)
        self.conv_layer_6 = nn.Conv2d(256,256,3,1)

        self.conv_layer_7 = nn.Conv2d(256,512,3,1)
        self.conv_layer_8 = nn.Conv2d(512,512,3,1)

        self.conv_layer_9 = nn.Conv2d(512,1024,3,1)
        self.conv_layer_10 = nn.Conv2d(1024,1024,3,1)

        self.upconv_layer_1 = nn.ConvTranspose2d(1024,512,2,2)

        self.De_conv_layer_1 = nn.Conv2d(1024,512,3,1)
        self.De_conv_layer_2 = nn.Conv2d(512,512,3,1)

        self.upconv_layer_2 = nn.ConvTranspose2d(512,256,2,2)

        self.De_conv_layer_3 = nn.Conv2d(512,256,3,1)
        self.De_conv_layer_4 = nn.Conv2d(256,256,3,1)

        self.upconv_layer_3 = nn.ConvTranspose2d(256,128,2,2)

        self.De_conv_layer_5 = nn.Conv2d(256,128,3,1)
        self.De_conv_layer_6 = nn.Conv2d(128,128,3,1)

        self.upconv_layer_4 = nn.ConvTranspose2d(128,64,2,2)

        self.De_conv_layer_7 = nn.Conv2d(128,64,3,1)
        self.De_conv_layer_8 = nn.Conv2d(64,64,3,1)

        self.final_conv_layer = nn.Conv2d(64,2,1,1)


    def forward(self, x):                               # 1x572x572
        c1_out = F.relu(self.conv_layer_1(x))           # 64x570x570
        c2_out = F.relu(self.conv_layer_2(c1_out))      # 64x568x568
        print(c1_out.shape, c2_out.shape)

        m1 =  nn.MaxPool2d(2, stride=2)
        p1_out = m1(c2_out)                             # 64x284x284
        print(p1_out.shape)

        c3_out = F.relu(self.conv_layer_3(p1_out))      # 128x282x282
        c4_out = F.relu(self.conv_layer_4(c3_out))      # 128x280x280
        print(c3_out.shape,c4_out.shape)

        m2 =  nn.MaxPool2d(2, stride=2)
        p2_out = m1(c4_out)                             # 128x140x140
        print(p2_out.shape)

        c5_out = F.relu(self.conv_layer_5(p2_out))      # 256x138x138
        c6_out = F.relu(self.conv_layer_6(c5_out))      # 256x136x136
        print(c5_out.shape,c6_out.shape)

        m3 =  nn.MaxPool2d(2, stride=2)
        p3_out = m1(c6_out)                             # 256x68x68
        print(p3_out.shape)

        c7_out = F.relu(self.conv_layer_7(p3_out))      # 512x66x66
        c8_out = F.relu(self.conv_layer_8(c7_out))      # 512x64x64
        print(c7_out.shape,c8_out.shape)

        m4 =  nn.MaxPool2d(2, stride=2)
        p4_out = m1(c8_out)                             # 512x32x32
        print(p4_out.shape)

        c9_out = F.relu(self.conv_layer_9(p4_out))      # 1024x30x30
        c10_out = F.relu(self.conv_layer_10(c9_out))    # 1024x28x28
        print(c9_out.shape,c10_out.shape)

        uc1_out = self.upconv_layer_1(c10_out)          # Upconv from 1024x28x28 to 512x56x56  - SWARNENDU
        print("size of the layer:", uc1_out.shape)

        cropped_tensor_1 = c8_out[:, :, 4:60, 4:60]     # Cropping from 512x64x64 to 512x56x56  - SWARNENDU
        combined_tensor_1 = torch.cat((cropped_tensor_1,uc1_out), dim=1)  # Concat across Channel Dimension, dim=1
        print("Combined tensor shape:", combined_tensor_1.shape)       # 1024x56x56

        dc_out_1 = F.relu(self.De_conv_layer_1(combined_tensor_1))
        dc_out_2 = F.relu(self.De_conv_layer_2(dc_out_1))
        print(dc_out_2.shape)

        uc2_out = self.upconv_layer_2(dc_out_2)
        print("size of the layer:", uc2_out.shape)

        cropped_tensor_2 = c6_out[:, :, 16:120, 16:120]
        combined_tensor_2 = torch.cat((cropped_tensor_2,uc2_out), dim=1)        # 512x104x104

        dc_out_3 = F.relu(self.De_conv_layer_3(combined_tensor_2))              # 256x102x102
        dc_out_4 = F.relu(self.De_conv_layer_4(dc_out_3))                       # 256x100x100
        print(dc_out_3.shape)      # 256x100x100

        uc3_out = self.upconv_layer_3(dc_out_4)
        print("size of the layer:", uc3_out.shape)    #128x200x200

        cropped_tensor_3 = c4_out[:, :, 40:240, 40:240]     #128x200x200
        combined_tensor_3 = torch.cat((cropped_tensor_3,uc3_out), dim=1)      # 26x200x200

        dc_out_5 = F.relu(self.De_conv_layer_5(combined_tensor_3))
        dc_out_6 = F.relu(self.De_conv_layer_6(dc_out_5))
        print(dc_out_3.shape)

        uc4_out = self.upconv_layer_4(dc_out_6)
        print("size of the layer:", uc4_out.shape)

        cropped_tensor_4 = c2_out[:, :, 88:480, 88:480]
        combined_tensor_4 = torch.cat((cropped_tensor_4,uc4_out), dim=1)

        dc_out_7 = F.relu(self.De_conv_layer_7(combined_tensor_4))              # 64x390x390
        dc_out_8 = F.relu(self.De_conv_layer_8(dc_out_7))                       # 64x388x388
        print(dc_out_8.shape)      # 64x388x388

        final_layer_out = F.softmax(self.final_conv_layer(dc_out_8),dim =1)    # 2x388x388

        return (final_layer_out)

model = UNet()

In [ ]:
# Initializing model
model = Unet().float().to(device)

epochs = 30

lr = 0.01

loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
train_loss = []
val_loss = []

Training

In [ ]:
for epoch in range(epochs):
    trainloss = 0
    valloss = 0
    c = 0
    for img, lab in tqdm(train_loader):
        optimizer.zero_grad()
        img = img.to(device)
        lab = lab.to(device)
        output = model(img)
        loss = loss_func(output, lab)
        loss.backward()
        optimizer.step()
        trainloss += loss.item()


        if epoch % 5 == 0 and c < 2:
            show([img[0], lab[0], output[0]])
            c += 1
    train_loss.append(trainloss/len(train_loader))



    for img, lab in tqdm(valid_loader):
        img = img.to(device)
        lab = lab.to(device)
        output = model(img)
        loss = loss_func(output, lab)
        valloss += loss.item()
    val_loss.append(valloss/len(valid_loader))

    print("epoch : {} ,train loss : {} ,valid loss : {} ".format(i,train_loss[-1],val_loss[-1]))

In [ ]:
plt.plot(train_loss,color='b',label='train loss')
plt.plot(val_loss,color='r',label = 'val_loss')
plt.legend()

In [ ]:
c = 0
for img, lab in valid_loader:
    img = img.to(device)
    label = lab.to(device)
    output = model(img)
    show([img[0],label[0],output[0]])
    if c>20:
        break
    c+=1